In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install "tensorflow_hub>=0.5.0"

In [0]:
import tensorflow_hub as hub
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
import glob,os, random
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [0]:
trainImage_data_gen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.1,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip=True,
    vertical_flip = True,
    validation_split = 0.1

)

testImage_data_gen = ImageDataGenerator(
    rescale = 1./255,
    validation_split =0.1
)

train_data_gen =trainImage_data_gen.flow_from_directory(
    'train',
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

val_data_gen = testImage_data_gen.flow_from_directory(
    'train',
    target_size=(224,224),
    batch_size = 16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

In [0]:
labels = (train_data_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

In [0]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL,input_shape=(224,224,3))

In [0]:
feature_extractor.trainable = False

In [0]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(11, activation='softmax')
])

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

In [0]:
# steps_per_epoch = training samples/batch_size
batch_size = 16
history = model.fit(train_data_gen, 
                      epochs=20, 
                      validation_data=val_data_gen,
                    )

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [0]:
test_x, test_y = val_data_gen.__getitem__(1)

prediction = model.predict(test_x)

plt.figure(figsize=(16, 16))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(prediction[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])

In [0]:
# save entire model to a HDF5 file
model.save('path/')

In [0]:
loaded_model = tf.keras.models.load_model('path/') 

In [0]:
!pip install tensorflowjs

In [0]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --weight_shard_size_bytes=4194304 path/ tfjsmodel/